In [12]:
from dhanhq import dhanhq, marketfeed
import talib
import numpy as np
import pandas as pd
import time
import datetime
import os
import logging

In [13]:
with open("token.txt", "r") as file:
    # Read the entire contents of the file
    token = file.read()
client_id = "1102249582"
access_token = token
dhan = dhanhq(client_id,access_token)

masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', nrows=1)
masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', usecols=masterlist.columns, low_memory=False)


logging.basicConfig(filename="logfile.txt", 
                    level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")


# Common function to place orders
def place_order(security_id, transaction_type, quantity):
    """
    Place an order using dhan API
    
    Args:
        security_id: Security ID to trade
        transaction_type: dhan.BUY or dhan.SELL
        quantity: Quantity to trade
    """
    print("Inside Place Order", security_id, transaction_type, quantity)
    # dhan.place_order(
    #     security_id=security_id,
    #     exchange_segment=dhan.NSE_FNO,
    #     transaction_type=transaction_type,
    #     quantity=quantity,
    #     order_type=dhan.MARKET,
    #     product_type=dhan.INTRA,
    #     price=0
    # )
    pass  # Placeholder since order is commented out


In [14]:
NIFTY_PREMIMUM = 26000

filtered_ce_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-11-04 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'CE') 
]
filtered_ce_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
153940,NSE,D,47680,OPTIDX,0,NIFTY-Nov2025-26000-CE,75.0,NIFTY 04 NOV 26000 CALL,2025-11-04 14:30:00,26000.0,CE,5.0,W,OP,NaN,NaN


In [15]:
filtered_pe_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-11-04 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'PE')
]

filtered_pe_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
153944,NSE,D,47691,OPTIDX,0,NIFTY-Nov2025-26000-PE,75.0,NIFTY 04 NOV 26000 PUT,2025-11-04 14:30:00,26000.0,PE,5.0,W,OP,NaN,NaN


In [22]:

SECURITY_ID = 47680

IS_ACTIVE = False
IS_OPEN_POSITION = False
quantity = 0
buy_price = 0     
stop_loss = 0

current_time = datetime.datetime.now()
current_second = current_time.second
time_str = current_time.strftime("%H:%M:%S")

current_date = datetime.datetime.now().strftime("%Y-%m-%d")

while IS_ACTIVE is False:
        current_second = datetime.datetime.now().second
        time.sleep(1)
        print("waiting to activate", current_second)
        if current_second == 0:
            IS_ACTIVE = True

while IS_ACTIVE is True and IS_OPEN_POSITION is False:
      current_second = datetime.datetime.now().second
      if current_second == 0:
            break
      print('Active',current_second)
      time.sleep(0.15)
      intraday = dhan.intraday_minute_data(str(SECURITY_ID), dhan.NSE_FNO, "OPTIDX", current_date, current_date, 1)

      print("intraday", intraday)
      intraday_df = pd.DataFrame(intraday['data']).tail(2)
      temp_list = []

    

      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]
      
      
     
      # if current_running_candle['close'] > previous_closed_candle['high']:
      #       print('Buy')
      #       length_of_previous_candle = round(previous_closed_candle['high'] - previous_closed_candle['low'])
      #       risk                      = 1000
      #       quantity                  = round(round(risk / length_of_previous_candle) / 75) * 75
      #       print('quantity',quantity)
            
      #       if quantity > 200:
      #             quantity = 200

      #       buy_price = int(current_running_candle['close'])
      #       stop_loss = int(previous_closed_candle['low'])

      #       place_order(SECURITY_ID, dhan.BUY, quantity)

      #       logging.info(f"BUY at {buy_price}")

      #       IS_OPEN_POSITION = True
      #       break;
      print('Buy')
      length_of_previous_candle = round(previous_closed_candle['high'] - previous_closed_candle['low'])
      risk                      = 1000
      quantity                  = 75 #round(round(risk / length_of_previous_candle) / 75) * 75
      print('quantity',quantity)
      
      if quantity > 200:
            quantity = 200

      buy_price = float(current_running_candle['close'])
      stop_loss = float(previous_closed_candle['low'])

      place_order(SECURITY_ID, dhan.BUY, quantity)

      logging.info(f"BUY at {buy_price}")
      print(f"BUY at {buy_price}")

      IS_OPEN_POSITION = True
      break




waiting to activate 48
waiting to activate 49
waiting to activate 50
waiting to activate 51
waiting to activate 52
waiting to activate 53
waiting to activate 54
waiting to activate 55
waiting to activate 57
waiting to activate 58
waiting to activate 59
waiting to activate 0
Active 1
intraday {'status': 'success', 'remarks': '', 'data': {'open': [188.0, 201.8, 199.7, 191.5, 185.2, 182.75, 189.75, 178.9, 180.9, 183.0, 188.3, 191.85, 197.05, 202.7, 202.15, 201.3, 204.2, 207.0, 198.55, 199.15, 190.3, 181.45, 180.5, 176.9, 174.05, 173.05, 174.75, 168.0, 171.0, 177.15, 172.55, 182.35, 186.2, 180.35, 169.7, 167.1, 171.9, 177.8, 177.45, 175.55, 179.1, 182.2, 186.0, 177.65, 175.05, 178.2, 184.9, 184.8, 187.15, 185.2, 182.45, 181.15, 180.55, 181.7, 183.2, 186.8, 186.9, 197.95, 196.4, 186.95, 185.9, 186.45, 185.7, 183.6, 184.85, 184.0, 191.5, 192.75, 195.05, 203.15, 201.9], 'high': [205.0, 203.4, 200.1, 195.15, 189.6, 191.8, 190.7, 182.25, 182.5, 189.7, 194.05, 198.5, 203.6, 206.2, 203.3, 207.5, 

In [ ]:

COST_TO_COST_SL = False
TRAILING_SL = False

while IS_OPEN_POSITION is True and quantity > 0:
      time.sleep(0.10)
      intraday = dhan.intraday_minute_data(str(SECURITY_ID), dhan.NSE_FNO, "OPTIDX", current_date, current_date, 1)
      print("waiting for sell")
      intraday_df = pd.DataFrame(intraday['data']).tail(2)

     
      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]

      # Stop loss
      if current_running_candle['close'] < stop_loss and COST_TO_COST_SL is False and TRAILING_SL is False:
            print("SELL")
            place_order(SECURITY_ID, dhan.SELL, quantity)
            print(f"SL hit {current_running_candle['close']}")
            IS_OPEN_POSITION = False

      # Trailing SL
      if current_running_candle['close'] < previous_closed_candle['low'] and COST_TO_COST_SL is False and TRAILING_SL is True:
            print("SELL")
            place_order(SECURITY_ID, dhan.SELL, quantity)
            print(f"Trailing SL hit {current_running_candle['close']}")
            IS_OPEN_POSITION = False

      # Shift sl to Cost to Cost
      if current_running_candle['close'] < buy_price +1 and COST_TO_COST_SL is True and TRAILING_SL is False:
            print("SELL")
            place_order(SECURITY_ID, dhan.SELL, quantity)
            print(f"CTC SL hit {current_running_candle['close']}")
            IS_OPEN_POSITION = False

            